In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [1]:
# Splitting a sequential dataset
#(split into multiple windows (short subset of a sentence) for each instance)
#(can create overlapping or non-overlapping windows)
#(use drop_remainder=True, works like padding="VALID")
#(flatten the dataset of windows, convert each window into a tensor/batch)
#(and shuffle)

In [ ]:
n_steps = 100
window_length = n_steps + 1   # steps and 1 character ahead as target
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
# 1 hot encode
dataset = dataset.map(
    lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

In [ ]:
# Tokenizing
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_text(text)   # text is dataset
max_id = len(tokenizer.word_index)   # number of characters

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
# Predict new text
X_new = preprocess(["text"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][1]   # 1st sentence, last character

In [ ]:
# Generate new texts
#(selecting a new word randomly with a log probability distribution)
#("temperature" controls the probability (higher=more random))
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return = tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

print(complete_text("t", temperature=0.2))

In [3]:
# Stateful RNN
#(set stateful=True in RNN layers)
#(RNN that preserves the hidden states between training batches)
#(the input sequence must be non-overlapping and not shuffled/sequential)
#(after each epoch, resets the state (using a callback))
#(stateful model can only predict batch of same size as training,
#avoid this by copy the weights to  stateless model)

In [ ]:
#(each batch is 1 window)
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

In [ ]:
#(proper batching)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = datasetflat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

In [ ]:
# Resetting after an epoch
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [4]:
# Sentimental analysis example model

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

In [3]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [4]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [5]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\hoang\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [6]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch                                      

In [7]:
from collections import Counter
vocab = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocab.update(list(review.numpy()))

In [8]:
vocab.most_common()[:3]

[(b'<pad>', 216693), (b'the', 61137), (b'a', 38564)]

In [9]:
vocab_size = 10000
truncated_vocab = [
    word for word, count in vocab.most_common()[:vocab_size]
]

In [10]:
words = tf.constant(truncated_vocab)
word_ids = tf.range(len(truncated_vocab), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [11]:
table.lookup(tf.constant([b"This movie was fantasticjalisj".split()]))

<tf.Tensor: id=101869, shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10983]], dtype=int64)>

In [12]:
#(alternatively use tft.compute_and_apply_vocabulary())

In [13]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [14]:
embed_size = 128

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential([
    Embedding(vocab_size + num_oov_buckets, embed_size,
             input_shape=[None]),
    GRU(128, return_sequences=True),
    GRU(128),
    Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
             metrics=["accuracy"])

In [ ]:
history = model.fit(train_set, epochs=5)

In [16]:
# Masking
#(mask all the inputs with word ID 0, ignores the padding tokens in the later layers)
#(done by specifying mask_zero=True for Embedding layer, and supports_masking=True for all layers
#receive the mask)
#(works automatically for sequential models,
#for more complex architecture explicitly computes and passes the mask)

In [17]:
# Manual masking
from tensorflow.keras.layers import Input, Lambda, Embedding, GRU, Dense
from tensorflow.keras import Model

K = keras.backend
inputs = Input(shape=[None])
mask = Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = GRU(128, return_sequences=True)(z, mask=mask)
z = GRU(128)(z, mask=mask)
outputs = Dense(1, activation="sigmoid")(z)
model = Model(inputs=[inputs], outputs=[outputs])

In [18]:
# Reusing pretrained embeddings
#(find a model here https://tfhub.dev/)

In [ ]:
import tensorflow_hub as hub

model = Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1',
                  dtype=tf.string, input_shape=[], output_shape=[50]),   # set trainable=True for training
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
             metrics=["accuracy"])

In [2]:
# Encoder Decoder architecture

In [ ]:
import tensorflow_addons as tfa

encoder_inputs = Input(shape=[None], dtype=np.int32)
decoder_inputs = Input(shape=[None], dtype=np.int32)
sequence_lengths = Input(shape=[], dtype=np.int32)

embeddings = Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = LSTMCell(512)
output_layer = Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                output_layout=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, inital_state=encoder_state,
    sequence_length=sequence_lengths)
y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
                   outputs=[y_proba])

In [ ]:
#(the original language sentence is fed into the encoder)
#(the encoder ouput and lag 1 of translated sentence/previous output are fed into the decoder)
#(the original sentence if reversed before fed so the sentence goes into the decoder in the right order)
#(use buckets of sentences of similar length and padding)
#(ignore outputs past end of sequence token)
#(in case of big vocabulary, only look at the output for correct word and random sample of incorrects,
#then approximate the loss to avoid calculating propba for every word (sampled softmax technique))
#(it is posisble to transition from using the target word as input to using the actual previous output during training,
#this is done with SceduledEmbeddingTrainingSampler)

In [2]:
# Bidirectional RNN
#(Causal/directional input works for time series)
#(bidirectional is preferable for NLP tasks)
#(runs 2 recurrent layers on the same input, 1 reads right to left, the other left to right)
#(use layers.Bidirectional() wrapper on a RNN layer)

In [3]:
# Beam Search
#(keeps track of a short list of possible predictions, recalculate the proba at each step)

In [ ]:
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_widt, output_layer=output_layer)
deocder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width)
outputs, _, _ = decoder(embedding_decoder, start_tokens=start_tokens, end_token=end_token,
                       initial_state=secoder_initial_state)